In [1]:
from datetime import date
import requests
import os.path
from os import path
import math
from tletools import TLE
from skyfield.api import Topos, load

today = date.today()
starlink_url = 'http://celestrak.com/NORAD/elements/starlink.txt'
tle_file = "../data/%s_starlink.tle"%today

if not path.exists(tle_file):
    r = requests.get(starlink_url, allow_redirects=True)
    open(tle_file, 'wb').write(r.content)
    print("Fetching",starlink_url )
else:
    print("Using",tle_file)

launches={
    19029:{'version': 0.9,'mission': 0},
    19074:{'version': 1.0,'mission': 1},
    20001:{'version': 1.0,'mission': 2},
    20006:{'version': 1.0,'mission': 3},
    20012:{'version': 1.0,'mission': 4},
    20019:{'version': 1.0,'mission': 5},
    20025:{'version': 1.0,'mission': 6},
    20035:{'version': 1.0,'mission': 7},
    20038:{'version': 1.0,'mission': 8},
}

DAY = 24*60*60
EARTH_RADIUS = 6371000.0
EARTH_MU = 3.986004418 * 10**14

norad={}

with open(tle_file , 'r') as tles:
    lines = tles.readlines()
    
objects=int(len(lines)/3)
#objects=1

print ("Processing %i objects"%objects)
  
for i in range(0,objects*3,3):
    tle_string = ''.join(lines[i:i+3])
    tle_lines = tle_string.strip().splitlines()
    tle = TLE.from_lines(*tle_lines)
    
    if '-' in tle.name:
        designator=tle.name.split('-')[0]
    else:
        designator=tle.name.split(' ')[0]
    launch=int(tle.int_desig[:5])
    mission=launches[launch]['mission']
    version=launches[launch]['version']
    piece=tle.int_desig[5:]
    
    if len(piece)==1:
        number=ord(piece)-64
        if number > 8:
           number-=1 
        
    if len(piece)==2:
        
        number0=ord(piece[0])-64
        
        number1=ord(piece[1])-64
        if number1 > 8:
           number1-=1 
        
        number=number0*25+ number1
    
    name="%s-%02d%02d"%(designator,mission,number)
    
    
    period = DAY / tle.n

    semi_major_axis = (((period / (math.pi*2))**2) * EARTH_MU) ** (1/3)
    ap_plus_pe = semi_major_axis * 2
    ap_minus_pe = tle.ecc * ap_plus_pe
    apogee = (ap_plus_pe + ap_minus_pe) / 2
    perigee = apogee - ap_minus_pe

    apogee -= EARTH_RADIUS
    perigee -= EARTH_RADIUS
  
    norad[int(tle.norad)]={'name':name,'mission':mission,'version':version,
                      'apogee':apogee,'perigee':perigee,'period':period}

print ("Processed %i objects"%len(norad))


Using ../data/2020-06-19_starlink.tle
Processing 545 objects
Processed 545 objects


In [8]:
satellites = load.tle_file(tle_file)
print('Loaded', len(satellites), 'objects')

other=[]
testing=[]
maneuver=[]
station=[]

for sat in satellites:
    satnum=sat.model.satnum
    name=norad[satnum]['name']
    apogee=norad[satnum]['apogee']
    perigee=norad[satnum]['perigee']
    version=norad[satnum]['version']
    
    if not 'STARLINK' in name:
        other.append(sat)
    else:
    
        if version < 1:
            testing.append(sat)
        else:
            if perigee > 500000:
                station.append(sat)
            else:
                maneuver.append(sat)
           
print ("Starlink on station:",len(station))
print ("Starlink manoeuvring:",len(maneuver))
print ("Starlink testing:",len(testing))
print ("Other objects:",len(other))
    

Loaded 545 objects
Starlink on station: 276
Starlink manoeuvring: 199
Starlink testing: 58
Other objects: 12


In [7]:
seattle = Topos('47.6062 N', '122.3321 W')
degrees=25.0

ts = load.timescale(builtin=True)
tsn = ts.now()
dtn = tsn.utc_datetime()
dtm = dtn.replace(hour=0,minute=0,second=0, microsecond=0)
dte = dtn.replace(hour=23,minute=59,second=59, microsecond=0)
tsm = ts.utc(dtm)
tse = ts.utc(dte)

for sat in station:
    satnum=sat.model.satnum
    name=norad[satnum]['name']
    times, events = sat.find_events(seattle, tm, te, altitude_degrees=degrees)

    for ti, event in zip(times, events):
        desc = ('rise', 'culm', 'set')[event]
        print(name,ti.utc_strftime('%H:%M:%S'), desc)

STARLINK-0101 07:08:36 rise
STARLINK-0101 07:10:49 culm
STARLINK-0101 07:13:03 set
STARLINK-0101 08:48:27 rise
STARLINK-0101 08:50:32 culm
STARLINK-0101 08:52:36 set
STARLINK-0101 10:28:49 rise
STARLINK-0101 10:30:41 culm
STARLINK-0101 10:32:33 set
STARLINK-0101 12:08:16 rise
STARLINK-0101 12:10:37 culm
STARLINK-0101 12:12:58 set
STARLINK-0102 06:44:10 rise
STARLINK-0102 06:46:01 culm
STARLINK-0102 06:47:53 set
STARLINK-0102 08:23:19 rise
STARLINK-0102 08:25:32 culm
STARLINK-0102 08:27:45 set
STARLINK-0102 10:03:49 rise
STARLINK-0102 10:05:38 culm
STARLINK-0102 10:07:26 set
STARLINK-0102 11:43:24 rise
STARLINK-0102 11:45:41 culm
STARLINK-0102 11:47:57 set
STARLINK-0102 13:23:33 rise
STARLINK-0102 13:25:06 culm
STARLINK-0102 13:26:38 set
STARLINK-0103 06:29:46 rise
STARLINK-0103 06:31:10 culm
STARLINK-0103 06:32:35 set
STARLINK-0103 08:08:16 rise
STARLINK-0103 08:10:34 culm
STARLINK-0103 08:12:51 set
STARLINK-0103 09:48:48 rise
STARLINK-0103 09:50:36 culm
STARLINK-0103 09:52:25 set
STAR